In [2]:
import pandas as pd
covid_vaccine_data = pd.read_csv(r"C:\Users\Raheyma Arshad\Desktop\INFO 5731 Term Project\RQ1 - Sentiment Analysis 02\Vaccine Tweets (With Location).csv")
del covid_vaccine_data['Unnamed: 0']
covid_vaccine_data = covid_vaccine_data[covid_vaccine_data.tweet != 'tweet']

In [ ]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from textblob import TextBlob
from googletrans import Translator
translator = Translator()
stop = stopwords.words('english')

# Data Cleaning
def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    # Translating the word
    try:
        text = TextBlob(word)
        word = (text.translate(to ='en')).text
    except:
        word = word
    return word

def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

def preprocess_tweet(tweet):
    porter_stemmer = PorterStemmer()
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # removing non-ascii characters i.e. characters with ascii value >= 128
    tweet = ''.join([w if ord(w) < 128 else ' ' for w in tweet])
    # Spelling correction
    tweet = str(TextBlob(tweet).correct())
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()
    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            # Removing stop words
            if word not in stop:
                processed_tweet.append(word)
    return ' '.join(processed_tweet)

In [ ]:
if __name__ == '__main__':
    clean_list = []
    #     tweet = "Facebook Censors President @realDonaldTrump: Takes Down Fox Interview Over Comments on Children and #Coronavirus Come on over to Parler, Gab, and MeWe, Mr. President! They do not censor conservatives. #ChinaVirus #COVID19 https://t.co/iFWCO3fXFC"
    #filepath = r'/home/junhua/PycharmProjects/TwitterDataAnalysis/datacollection/Useful_data/useful_data.csv'
    #dataframe = pd.read_csv(filepath)
    #     print(dataframe.head())
    df_list = covid_vaccine_data['tweet'].values.tolist()
    df_sublists = chunkIt(df_list, 20)
    for row in df_sublists[0]:
        if not row[1]:
            print("The string is empty")
        elif len(row[1]) < 4:
            print("The string is too short")
        else:
            clean_tweet = preprocess_tweet(row[1])
            new_row = [row[0], clean_tweet, row[2], row[3], row[4]]
            print(new_row)
            clean_list.append(new_row)
    new_df = pd.DataFrame(clean_list, columns=['Index', 'Tweet', 'Relationship_Date_UTC', 'Hashtags_in_Tweet', 'Location'])
    new_df.to_csv(r'C:\Users\Raheyma Arshad\Desktop\Paper Data.csv')